In [1]:
import numpy as np
import tensorflow as tf
import nltk
import random
import json
from nltk.stem import WordNetLemmatizer

In [2]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Utente\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Utente\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
with open('train_data.json', 'r') as file:
    intents = json.load(file)

In [4]:
lemmatizer=WordNetLemmatizer()
words=[]   #list of all the words
tags=[] #tags
documents=[] #(words_of_the_tag,tag)
ignoreLetters=['?','!','.',',']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)
        documents.append((wordList, intent['tag']))
        if intent['tag'] not in tags:
            tags.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]
words = sorted(set(words))

tags = sorted(set(tags))


training = []
outputEmpty = [0] * len(tags)

for document in documents:
    bag = []
    wordPatterns = document[0]  #words of the tag
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(outputEmpty)
    outputRow[tags.index(document[1])] = 1
    training.append(bag + outputRow)

random.shuffle(training)
training = np.array(training)

trainX = training[:, :len(words)]
trainY = training[:, len(words):]


model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[0]),), activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(trainX), np.array(trainY), epochs=200, batch_size=5, verbose=1)

print('Done')


Epoch 1/200


81/81 [==============================] - 3s 4ms/step - loss: 3.6036 - accuracy: 0.0420
Epoch 2/200
81/81 [==============================] - 0s 4ms/step - loss: 3.4552 - accuracy: 0.1086
Epoch 3/200
81/81 [==============================] - 0s 4ms/step - loss: 3.2389 - accuracy: 0.1704
Epoch 4/200
81/81 [==============================] - 0s 4ms/step - loss: 2.9102 - accuracy: 0.2667
Epoch 5/200
81/81 [==============================] - 0s 4ms/step - loss: 2.6571 - accuracy: 0.3136
Epoch 6/200
81/81 [==============================] - 0s 3ms/step - loss: 2.4831 - accuracy: 0.3457
Epoch 7/200
81/81 [==============================] - 0s 3ms/step - loss: 2.2919 - accuracy: 0.3753
Epoch 8/200
81/81 [==============================] - 0s 4ms/step - loss: 2.0278 - accuracy: 0.4667
Epoch 9/200
81/81 [==============================] - 0s 4ms/step - loss: 1.9142 - accuracy: 0.4667
Epoch 10/200
81/81 [==============================] - 0s 4ms/step - loss: 1.7526 - accuracy: 0.5012
Epoch 

In [5]:
lemmatizer = WordNetLemmatizer()

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words (sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class (sentence):
    bow = bag_of_words (sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': tags [r[0]], 'probability': str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice (i['responses'])
            break
    return result

print("GO! Bot is running!")

while True:
    message = input(">>")
    if message=='q':
      break
    ints = predict_class (message)
    res = get_response (ints, intents)
    print (res)

GO! Bot is running!
1/1 [==============================] - 0s 257ms/step
Hello!
1/1 [==============================] - 0s 81ms/step
Hello!
1/1 [==============================] - 0s 70ms/step
I am your helper
1/1 [==============================] - 0s 60ms/step
College students
1/1 [==============================] - 0s 62ms/step
I'm Mind Reader
1/1 [==============================] - 0s 77ms/step
College students
1/1 [==============================] - 0s 81ms/step
College students
1/1 [==============================] - 0s 62ms/step
College students
1/1 [==============================] - 0s 58ms/step
<a target="_blank" href="ADD YOU GOOGLE MAP LINK HERE"> here</a>
